# Digital Audit Techniques: Analysing Financial Records for Journal Entry Testing and Anomaly Detection

--------------

## Table of Contents

1. [Introduction](#Introduction)

2. [Data Import](#Data_Import)

3. [Exploratory Data Analysis](#Exploratory_Data_Analysis)

4. [Data Preparation](#Data_Preparation)

   4.1 [Conversion of data type](#Conversion_of_Data_Types)

   4.2 [Data Filtering](#Data_Filtering)
   
5. [Modeling](#Modeling)
   
   5.1 [Sunday Entries](#Sunday_Entries)

   5.2 [Cash Minus Inspection](#Cash_Minus_Inspection)

   5.3 [Checking for duplicates](#Checking_for_duplicates)

   5.4 [Checking the pre-decimal digits for equal (uniform) distribution](#Checking_the_pre-decimal_digits_for_equal_distribution)

   5.5 [Checking the leading digit for a Benford distribution](#Checking_the_leading_digit_for_a_Benford_distribution)
   
6. [Evaluation](#Evaluation)




--------------

## 1. Introduction <a id="Introduction"></a>

This project is about applying digital audit techniques to synthetic datasets that simulate real-world financial records. The goal is to conduct a Journal Entry Test to evaluate the integrity of financial statements.

--------------

## 2. Data Import <a id="Data_Import"></a>

We start by loading the necessary libraries for subsequent analysis.

[Code section 1]

In [1]:
# IMPORTING LIBRARIES

import datetime
import math
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import plotly.io as pio
import plotly.express as px

We also load the datasets, which are stored in CSV files. The datasets provided include:
- Accounts Payable Data `accountspayable.csv`: Vendor master data, containing details about suppliers or service providers with outstanding liabilities.
- Accounts Receivable Data `accountsreceivable.csv`: Customer master data, with basic information about the company's customers.
- Bank Accounts Data `bankaccounts.csv`: Bank master data, detailing the company’s financial institutions and accounts.
- Transactions Data `transactions.csv`: Posting records that reflect movements in the company’s accounts.

[Code section 2]

In [2]:
# IMPORTING DATASETS

df_accounts_payable = pd.read_csv('accountspayable.csv', sep = ";", encoding = "UTF-8")
df_accounts_receivable = pd.read_csv('accountsreceivable.csv', sep = ";", encoding = "UTF-8")
df_bank_accounts = pd.read_csv('bankaccounts.csv', sep = ";", encoding = "UTF-8")
df_transactions = pd.read_csv('transactions.csv', sep = ";", encoding = "UTF-8")

--------------

## 3. Exploratory Data Analysis <a id="Exploratory_Data_Analysis"></a>

The datasets that we have loaded are saved in variables of the type DataFrame, as shown below using the `type` function.

The DataFrame is a two-dimensional data structure that can store data of different types in columns provided by the pandas library.

[Code section 3]

In [3]:
# TYPE FUNCTION

type(df_accounts_payable)
type(df_accounts_receivable)
type(df_bank_accounts)
type(df_transactions)

pandas.core.frame.DataFrame

Every DataFrame is characterized by a number of rows and columns. The rows are the individual records, while the columns are the attributes of the records. The columns are the variables that we can use to analyze the data. The `shape` attribute of a DataFrame returns the number of rows and columns.

[Code section 4]

In [4]:
# SHAPE ATTRIBUTE

print("The shape of the \"Accounts Payable\" data-frame is:", df_accounts_payable.shape)
print("The shape of the \"Accounts Receivable\" data-frame is:", df_accounts_receivable.shape)
print("The shape of the \"Bank Accounts\" data-frame is:", df_bank_accounts.shape)
print("The shape of the \"Transactions\" data-frame is:", df_transactions.shape)

The shape of the "Accounts Payable" data-frame is: (21, 7)
The shape of the "Accounts Receivable" data-frame is: (21, 7)
The shape of the "Bank Accounts" data-frame is: (31, 9)
The shape of the "Transactions" data-frame is: (107, 12)


Now, we will display the first few rows of the DataFrames using the `head(n)` method, which returns the first n rows of the DataFrame. The default value of n is 5.

[Code section 5]

In [5]:
# HEAD METHOD

print("The first 3 rows of the \"Accounts Payable\" data-frame are:")
display(df_accounts_payable.head(3))

print("The first 3 rows of the \"Accounts Receivable\" data-frame are:")
display(df_accounts_receivable.head(3))

print("The first 3 rows of the \"Bank Accounts\" data-frame are:")
display(df_bank_accounts.head(3))

print("The first 3 rows of the \"Transactions\" data-frame are:")
display(df_transactions.head(3))

The first 3 rows of the "Accounts Payable" data-frame are:


,Creditor_ID,Firm,Street,HouseNr.,City,Postcode,Country
0,1,Workflow GmbH,Berlinerstraße,12,Berlin,10115,Germany
1,2,Consulting_solutions,Elysees,11,Montord,33452,France
2,3,Consulting_regulations,Prugne,73,Cesset,24532,France


The first 3 rows of the "Accounts Receivable" data-frame are:


,Creditor_ID,Firm,Street,HouseNr.,City,Postcode,Country
0,1,Workflow GmbH,Berlinerstraße,12,Berlin,10115,Germany
1,2,Consulting_solutions,Elysees,11,Montord,33452,France
2,3,Consulting_regulations,Prugne,73,Cesset,24532,France


The first 3 rows of the "Bank Accounts" data-frame are:


,Bank_ID,Creditor_ID,Bankname,CountryKey,VerifCode,BankKey,SectorCode,BankAccNr,CheckDigit
0,1,1,Sparkasse,DE,44,85708525,-,9092573071,-
1,2,1,Sparkasse,DE,27,34010441,-,1517137596,-
2,3,2,BGFI,FR,29,53954085,12914,93097,31


The first 3 rows of the "Transactions" data-frame are:


,Booking_ID,Debitor_ID,Creditor_ID,Product_ID,AccNr_Debit,AccNr_Credit,Value,Date,Time,PaymentTarget,Paid,Reminders
0,1,3,-,8,1200,8000,"33,40",01.01.2018,11:00,15.01.2018,no,1
1,2,5,-,4,1200,8000,"122,00",01.01.2018,20:56,15.01.2018,yes,-
2,3,18,-,10,1200,8000,"22,44",03.01.2018,17:31,17.01.2018,yes,-


To gain a better understanding of the data, we will use the `describe()` method, which returns descriptive statistics that summarize statistics of numerical columns of the dataset.

The `describe()` method provides information on:
- the count of non-null values in each numerical column,
- the mean of each numerical column,
- the standard deviation of each numerical column,
- the minimum values of each numerical column,
- the 25th percentiles of each numerical column.
- the 50th percentiles of each numerical column.
- the 75th percentiles of each numerical column.
- the maximum values of each numerical column.


[Code section 6]

In [6]:
# DESCRIBE METHOD

print("The main statistics of the \"Accounts Payable\" data-frame are:")
display(df_accounts_payable.describe())

print("The main statistics of the \"Accounts Receivable\" data-frame are:")
display(df_accounts_receivable.describe())

print("The main statistics of the \"Bank Accounts\" data-frame are:")
display(df_bank_accounts.describe())

print("The main statistics of the \"Transactions\" data-frame are:")
display(df_transactions.describe())

The main statistics of the "Accounts Payable" data-frame are:


,Creditor_ID,HouseNr.,Postcode
count,21.000000,21.000000,21.000000
mean,11.000000,40.095238,53375.380952
std,6.204837,25.125495,29396.611579
min,1.000000,2.000000,10115.000000
25%,6.000000,14.000000,24532.000000
50%,11.000000,42.000000,54677.000000
75%,16.000000,59.000000,78654.000000
max,21.000000,83.000000,94469.000000


The main statistics of the "Accounts Receivable" data-frame are:


,Creditor_ID,HouseNr.,Postcode
count,21.000000,21.000000,21.000000
mean,11.000000,40.095238,53375.380952
std,6.204837,25.125495,29396.611579
min,1.000000,2.000000,10115.000000
25%,6.000000,14.000000,24532.000000
50%,11.000000,42.000000,54677.000000
75%,16.000000,59.000000,78654.000000
max,21.000000,83.000000,94469.000000


The main statistics of the "Bank Accounts" data-frame are:


,Bank_ID,Creditor_ID,VerifCode,BankKey,BankAccNr
count,31.000000,31.000000,31.000000,3.100000e+01,3.100000e+01
mean,16.000000,10.387097,61.838710,5.597389e+07,2.186200e+09
std,9.092121,6.195038,27.427598,2.377010e+07,3.363191e+09
min,1.000000,1.000000,13.000000,1.330508e+07,1.944100e+04
25%,8.500000,5.500000,33.500000,3.781969e+07,5.980650e+04
50%,16.000000,10.000000,63.000000,5.703530e+07,9.190400e+04
75%,23.500000,15.500000,90.000000,7.964018e+07,4.076312e+09
max,31.000000,21.000000,99.000000,9.948417e+07,9.092573e+09


The main statistics of the "Transactions" data-frame are:


,Booking_ID,AccNr_Debit,AccNr_Credit
count,107.000000,107.000000,107.000000
mean,54.000000,2039.252336,5514.018692
std,31.032241,1215.525122,3171.704161
min,1.000000,1000.000000,1000.000000
25%,27.500000,1200.000000,1600.000000
50%,54.000000,1200.000000,8000.000000
75%,80.500000,3200.000000,8000.000000
max,107.000000,5100.000000,8000.000000


We proceed by providing a summary of the DataFrames using the `info()` method, which returns information about the DataFrame, including the data type of each column, the number of non-null values in each column, and the memory usage of the DataFrame.

[Code section 7]

In [7]:
# INFO METHOD

print("The information of the \"Accounts Payable\" data-frame is:")
display(df_accounts_payable.info())

print("The information of the \"Accounts Receivable\" data-frame is:")
display(df_accounts_receivable.info())

print("The information of the \"Bank Accounts\" data-frame is:")
display(df_bank_accounts.info())

print("The information of the \"Transactions\" data-frame is:")
display(df_transactions.info())

The information of the "Accounts Payable" data-frame is:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Creditor_ID  21 non-null     int64 
 1   Firm         21 non-null     object
 2   Street       21 non-null     object
 3   HouseNr.     21 non-null     int64 
 4   City         21 non-null     object
 5   Postcode     21 non-null     int64 
 6   Country      21 non-null     object
dtypes: int64(3), object(4)
memory usage: 1.3+ KB


None

The information of the "Accounts Receivable" data-frame is:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Creditor_ID  21 non-null     int64 
 1   Firm         21 non-null     object
 2   Street       21 non-null     object
 3   HouseNr.     21 non-null     int64 
 4   City         21 non-null     object
 5   Postcode     21 non-null     int64 
 6   Country      21 non-null     object
dtypes: int64(3), object(4)
memory usage: 1.3+ KB


None

The information of the "Bank Accounts" data-frame is:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Bank_ID      31 non-null     int64 
 1   Creditor_ID  31 non-null     int64 
 2   Bankname     31 non-null     object
 3   CountryKey   31 non-null     object
 4   VerifCode    31 non-null     int64 
 5   BankKey      31 non-null     int64 
 6   SectorCode   31 non-null     object
 7   BankAccNr    31 non-null     int64 
 8   CheckDigit   31 non-null     object
dtypes: int64(5), object(4)
memory usage: 2.3+ KB


None

The information of the "Transactions" data-frame is:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Booking_ID     107 non-null    int64 
 1   Debitor_ID     107 non-null    object
 2   Creditor_ID    107 non-null    object
 3   Product_ID     107 non-null    object
 4   AccNr_Debit    107 non-null    int64 
 5   AccNr_Credit   107 non-null    int64 
 6   Value          107 non-null    object
 7   Date           107 non-null    object
 8   Time           107 non-null    object
 9   PaymentTarget  107 non-null    object
 10  Paid           107 non-null    object
 11  Reminders      107 non-null    object
dtypes: int64(3), object(9)
memory usage: 10.2+ KB


None

We proceed by displaying the columns of the DataFrames using the `columns` attribute which returns the column labels of the DataFrame.

[Code section 8]

In [8]:
# COLUMNS ATTRIBUTE

print("The columns of the \"Accounts Payable\" data-frame are:")
display(df_accounts_payable.columns)

print("The columns of the \"Accounts Receivable\" data-frame are:")
display(df_accounts_receivable.columns)

print("The columns of the \"Bank Accounts\" data-frame are:")
display(df_bank_accounts.columns)

print("The columns of the \"Transactions\" data-frame are:")
display(df_transactions.columns)

The columns of the "Accounts Payable" data-frame are:


Index(['Creditor_ID', 'Firm', 'Street', 'HouseNr.', 'City', 'Postcode',
       'Country'],
      dtype='object')

The columns of the "Accounts Receivable" data-frame are:


Index(['Creditor_ID', 'Firm', 'Street', 'HouseNr.', 'City', 'Postcode',
       'Country'],
      dtype='object')

The columns of the "Bank Accounts" data-frame are:


Index(['Bank_ID', 'Creditor_ID', 'Bankname', 'CountryKey', 'VerifCode',
       'BankKey', 'SectorCode', 'BankAccNr', 'CheckDigit'],
      dtype='object')

The columns of the "Transactions" data-frame are:


Index(['Booking_ID', 'Debitor_ID', 'Creditor_ID', 'Product_ID', 'AccNr_Debit',
       'AccNr_Credit', 'Value', 'Date', 'Time', 'PaymentTarget', 'Paid',
       'Reminders'],
      dtype='object')

The `values` attribute returns a Numpy representation of the DataFrame, which is a two-dimensional array of the data in the DataFrame. We will display the values of one of the DataFrames using the `values` attribute.

[Code section 9]

In [9]:
# VALUES ATTRIBUTE

print("The values of the \"Accounts Payable\" data-frame are:")
display(df_accounts_payable.values)

The values of the "Accounts Payable" data-frame are:


array([[1, 'Workflow GmbH', 'Berlinerstraße', 12, 'Berlin', 10115,
        'Germany'],
       [2, 'Consulting_solutions', 'Elysees', 11, 'Montord', 33452,
        'France'],
       [3, 'Consulting_regulations', 'Prugne', 73, 'Cesset', 24532,
        'France'],
       [4, 'IT_Solutions', 'Maximilianstraße', 11, 'Munich', 80331,
        'Germany'],
       [5, 'Governance GmbH', 'Frankstraße', 75, 'Munich', 80332,
        'Germany'],
       [6, 'Business_Partnership', 'Rue de Grivats', 60, 'Lyon', 53522,
        'France'],
       [7, 'Clera', 'Chemin ', 59, 'Paris', 75000, 'France'],
       [8, 'Carbon', 'Hornerstraße', 52, 'Hamburg', 20090, 'Germany'],
       [9, "Bon app'", 'Gouzol', 2, 'Beauron', 67976, 'France'],
       [10, 'Bottom-Dollar Marketse', 'Robin', 51, 'Lyon', 53454,
        'France'],
       [11, 'Kraft GmbH', 'Theodorstraße', 70, 'Deggendorf', 94469,
        'Germany'],
       [12, 'EDF', "de L'Est", 14, 'Marseille', 78654, 'France'],
       [13, 'Centro comercial', 'Curi

--------------

## 4. Data Preparation <a id="Data_Preparation"></a>

### 4.1 Conversion of data types <a id="Conversion_of_Data_Types"></a>

After the data import, a variable is not always automatically assigned to the appropriate object class, which can become a problem, especially when functions require a certain object class.

[Code section 10]

In [10]:
# CONVERTING DATA TYPES

df_transactions['Value'] = df_transactions['Value'].str.replace(',', '.').astype(float)

As it will be needed in the following steps, we will convert the 'Date' column of the data frame into the day of the week using the `day_name()` function and added it as a new column to the Transactions data-frame.

[Code section 11]

In [11]:
# CONVERTING DATA TYPES

df_transactions['Date'].dtypes
df_transactions['Date'] = pd.to_datetime(df_transactions['Date'], format = '%d.%m.%Y')
df_transactions['DayOfTheWeek'] = df_transactions['Date'].dt.day_name()
df_transactions['DayOfTheWeek']

0         Monday
1         Monday
2      Wednesday
3       Thursday
4       Thursday
         ...    
102    Wednesday
103     Thursday
104       Friday
105     Saturday
106       Sunday
Name: DayOfTheWeek, Length: 107, dtype: object

The Reminders column shows that the first category is "-". In the following line of code, all "-" elements are substituted with "0" and the command .astype(str) first converts the variable to a string variable, then to an integer using .astype(int).

[Code section 12]

In [12]:
# CONVERTING DATA TYPES

df_transactions['Reminders'] = df_transactions['Reminders'].replace('-', '0')
df_transactions['Reminders'] = df_transactions['Reminders'].astype(str).astype(int)

### 4.2 Data Filtering <a id="Data_Filtering"></a>

We proceed by creating a new data-frame with the columns `AccNr_Debit` and `AccNr_Credit` equal to $1000$, in order to filter the transactions from the ones that not part of the cash account.

[Code section 13]

In [13]:
# FILTERING DATA

display(df_transactions.AccNr_Debit == 1000)
df_cash_book = df_transactions[(df_transactions.AccNr_Debit == 1000) | (df_transactions.AccNr_Credit == 1000)]

display(df_cash_book.head())

0      False
1      False
2      False
3       True
4      False
       ...  
102    False
103    False
104     True
105    False
106    False
Name: AccNr_Debit, Length: 107, dtype: bool

,Booking_ID,Debitor_ID,Creditor_ID,Product_ID,AccNr_Debit,AccNr_Credit,Value,Date,Time,PaymentTarget,Paid,Reminders,DayOfTheWeek
3,4,9,-,5,1000,8000,182.40,2018-01-04,00:26,-,yes,0,Thursday
5,6,-,11,-,5100,1000,193.33,2018-01-05,13:14,-,yes,0,Friday
6,7,13,-,5,1000,8000,364.80,2018-01-06,10:57,-,yes,0,Saturday
9,10,-,11,-,5100,1000,245.56,2018-01-07,15:21,-,yes,0,Sunday
13,14,18,-,1,1000,8000,1123.12,2018-01-09,22:23,-,yes,0,Tuesday


We proceed by merging the `df_accounts_payable` and `df_bank_accounts` data-frames on the `Creditor_ID` column.

[Code section 14]

In [14]:
# MERGING DATA

print(df_accounts_receivable.columns)
print(df_bank_accounts.columns)

df_accounts_receivable_banks = df_bank_accounts.merge(df_accounts_receivable, how = 'inner', left_on = 'Creditor_ID', right_on = 'Creditor_ID')
df_accounts_receivable_banks.head()

Index(['Creditor_ID', 'Firm', 'Street', 'HouseNr.', 'City', 'Postcode',
       'Country'],
      dtype='object')
Index(['Bank_ID', 'Creditor_ID', 'Bankname', 'CountryKey', 'VerifCode',
       'BankKey', 'SectorCode', 'BankAccNr', 'CheckDigit'],
      dtype='object')


,Bank_ID,Creditor_ID,Bankname,CountryKey,VerifCode,BankKey,SectorCode,BankAccNr,CheckDigit,Firm,Street,HouseNr.,City,Postcode,Country
0,1,1,Sparkasse,DE,44,85708525,-,9092573071,-,Workflow GmbH,Berlinerstraße,12,Berlin,10115,Germany
1,2,1,Sparkasse,DE,27,34010441,-,1517137596,-,Workflow GmbH,Berlinerstraße,12,Berlin,10115,Germany
2,3,2,BGFI,FR,29,53954085,12914,93097,31,Consulting_solutions,Elysees,11,Montord,33452,France
3,4,3,BGFI,FR,92,29603999,98110,90295,6,Consulting_regulations,Prugne,73,Cesset,24532,France
4,5,3,Banco Frances,FR,63,67849978,16309,91904,9,Consulting_regulations,Prugne,73,Cesset,24532,France


--------------

## 5. Modeling <a id="Modeling"></a>

### 5.1 Sunday Entries <a id="Sunday_Entries"></a>

We proceed by filtering the transactions that were posted on a Sunday.

[Code section 15]

In [15]:
# RETRIEVING WEEKEND BOOKINGS

df_transactions_sundays = df_transactions[df_transactions['DayOfTheWeek'] == "Sunday"]
print("The number of books entered on Sundays is", df_transactions_sundays.shape[0], "out of a total of", df_transactions.shape[0], "books.")

The number of books entered on Sundays is 15 out of a total of 107 books.


### 5.2 Cash Minus Inspection <a id="Cash_Minus_Inspection"></a>

Now, we check if the cash account has ever been negative.

[Code section 16]

In [16]:
# CALCULATING CUMULATIVE SUM

df_cash_book = df_cash_book.reset_index(drop = True)

c = 0

for i in range(0, len(df_cash_book)):

    entry = df_cash_book.Value[i]
    if df_cash_book.AccNr_Credit[i] == 1000:
        entry = entry * (-1)

    c = c + entry
    df_cash_book.loc[i, 'Cumulative_Sum'] = c

print("Cash register was ", (df_cash_book.Cumulative_Sum < 0).sum(), "times in minus.")

df_cash_book[df_cash_book.Cumulative_Sum < 0]

Cash register was  5 times in minus.


,Booking_ID,Debitor_ID,Creditor_ID,Product_ID,AccNr_Debit,AccNr_Credit,Value,Date,Time,PaymentTarget,Paid,Reminders,DayOfTheWeek,Cumulative_Sum
1,6,-,11,-,5100,1000,193.33,2018-01-05,13:14,-,yes,0,Friday,-10.93
5,20,-,11,-,5100,1000,1500.00,2018-01-14,16:33,-,yes,0,Sunday,-268.57
6,22,2,-,8,1000,8000,33.40,2018-01-16,22:46,-,yes,0,Tuesday,-235.17
7,25,11,-,2,1000,8000,17.56,2018-01-18,03:32,-,yes,0,Thursday,-217.61
8,34,10,-,9,1000,8000,3.80,2018-01-28,16:24,-,yes,0,Sunday,-213.81


### 5.3 Checking for duplicates <a id="Checking_for_duplicates"></a>

We proceed by checking for duplicates in the `df_accounts_payable` data-frame.

[Code section 17]

In [17]:
# CHECKING FOR DUPLICATES

BankAccNr_duplicates = df_accounts_receivable_banks.loc[df_accounts_receivable_banks.BankAccNr.duplicated(), 'BankAccNr']
print("The number of duplicates in the Accounts Receivable Bank data-frame is", BankAccNr_duplicates.shape[0])
print("The index of the duplicate is:" , BankAccNr_duplicates.index[0])

# # Another way of showing the index of the duplicate
# df_accounts_receivable_banks['BankAccNr'].isin(BankAccNr_duplicates)

# # Alternative
# df_accounts_receivable_banks[df_accounts_receivable_banks.BankAccNr.duplicated()]
# # Note: indicates only the duplicate!


The number of duplicates in the Accounts Receivable Bank data-frame is 1
The index of the duplicate is: 30


### 5.4 Checking the pre-decimal digits for equal (uniform) distribution <a id="Checking_the_pre-decimal_digits_for_equal_distribution"></a>

We proceed by checking the pre-decimal digits for equal (uniform) distribution, using the Chi-square test.

[Code section 18]

In [18]:
# CHECKING THE PRE-DECIMAL DIGITS FOR EQUAL (UNIFORM) DISTRIBUTION

Integer = df_transactions.Value.astype(int)

DigitBeforeComma = pd.DataFrame([int(str(x)[-1]) for x in Integer])

# # Alternative way of doing that
# DigitBeforeComma = pd.DataFrame(Integer).astype(str)
# DigitBeforeComma['DigitBeforeComma'] = DigitBeforeComma['Value'].str[-1]

# # Another alternative way of doing that
# DigitBeforeComma = pd.DataFrame(map(int, [str(x)[-1] for x in Integer]))

H_DigitBeforeComma = DigitBeforeComma.value_counts()
H_DigitBeforeComma = H_DigitBeforeComma.sort_index()
print(H_DigitBeforeComma)

sp.stats.chisquare(H_DigitBeforeComma)

0
0     2
1     5
2    23
3    31
4     3
5     6
6    12
7    11
8     3
9    11
Name: count, dtype: int64


Power_divergenceResult(statistic=np.float64(76.08411214953271), pvalue=np.float64(9.65032885139533e-13))

The null hypothesis can be rejected at any significance level lower than $\alpha = 0.05$. The output of the Chi Square test shows that our p-value is of $9.65e-13$, hence even smaller than the more restrictive threshold of $0.01$. Therefore, we see infimal empirical evidence in favour of the null hypothesis (that is, the precomma digits of the booking values follow the equal distribution) and so we reject it. 

### 5.5 Checking the leading digit for a Benford distribution <a id="Checking_the_leading_digit_for_a_Benford_distribution"></a>

We proceed by visualizing the Benford distribution.

[Code section 19]

In [19]:
# VISUALIZATION OF THE BENFORD DISTRIBUTION PROBABILITIES

numbers = np.arange(1,10)
Probs = []
for x in numbers: Probs.append(math.log10(1 + 1/x))
np.arange(len(numbers))

Benford_distribution_histogram = px.histogram(x = numbers, y = Probs, nbins = 9, histnorm ='probability density', category_orders=dict(day=["1", "2", "3", "4", "5", "6", "7", "8", "9"]))
Benford_distribution_histogram.update_layout(bargap=0.1)
Benford_distribution_histogram.update_xaxes(title_text='Digit')
Benford_distribution_histogram.update_yaxes(title_text='Probability')
Benford_distribution_histogram.update_layout(title_text='Benford Distribution Probabilities')
display(Benford_distribution_histogram)

Ultimately, we will check the leading digit for a Benford distribution, using the Chi-square test.

[Code section 20]

In [20]:
# CHECKING THE FIRST DIGITS FOR BENFORD DISTRIBUTION

# Creating a data-frame with the first digits of the integer values
FirstDigit = pd.DataFrame(map(int, [str(x)[0] for x in Integer]))

H_FirstDigit = pd.DataFrame(FirstDigit.value_counts().sort_index())
H_FirstDigit.reset_index(inplace = True)

H_FirstDigit.columns = ['FirstDigit', 'Frequency']
H_FirstDigit.loc[len(H_FirstDigit)] = [9, 0]
H_FirstDigit.set_index('FirstDigit', inplace = True)

print("Below, the first digits and their frequencies are shown:")
display(H_FirstDigit)

# Defining the expected frequencies
Probs = []
for x in numbers: Probs.append(math.log10(1 + 1/x))
for i in range(0, len(Probs)):
    Probs[i] = Probs[i] * len(FirstDigit)

# creating a data-frame with the expected frequencies
Probs = pd.DataFrame(Probs)
Probs.columns = ['Expected_Frequency']
Probs.index.name = 'FirstDigit'

print("Below, the expected frequencies are shown:")
display(Probs)

# Performing the Chi-square test
sp.stats.chisquare(f_obs = H_FirstDigit["Frequency"], f_exp = Probs["Expected_Frequency"])

Below, the first digits and their frequencies are shown:


,Frequency
FirstDigit,
1,30
2,12
3,15
4,15
5,12
6,9
7,10
8,4
9,0


Below, the expected frequencies are shown:


,Expected_Frequency
FirstDigit,
0,32.210210
1,18.841765
2,13.368445
3,10.369371
4,8.472393
5,7.163306
6,6.205138
7,5.473320
8,4.896051


Power_divergenceResult(statistic=np.float64(14.45619805497751), pvalue=np.float64(0.0706227444346869))

The output shows that the null hypothesis cannot be rejected at a significance level of $5\%$ since the p-value of the chi-square test is equal to $0.07062$. Thus, at the $5\%$ significance level no deviation of the distribution of the first digit from the Benford distribution can be substantiated, although at the $10%$ significance level it can be.

--------------

## 6. Evaluation <a id="Evaluation"></a>

The initial task was to use alternative methods of digital data analysis when auditing annual financial statements. The following conclusions can be drawn from the available synthetic data, i.e. artificially generated sample data:

1. Weekend bookings: 15 bookings took place on a Sunday; the corresponding booking records can be viewed using df_transactions_sundays and should be subjected to further investigations.
2. Cash minus inspection: The cash was five times in the minus.
3. Validation of duplicates: A duplicate was identified (Workflow GmbH Berlin). The data must be cleaned up and consolidated.
